<a href="https://colab.research.google.com/github/amukund8/AML/blob/main/CrossValidationPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define a simple convolutional neural network model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Instantiate the model, loss function, and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Set training parameters
num_epochs = 5
batch_size = 64

# Download and preprocess the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {average_loss}")

print("Training complete.")


100%|██████████| 9912422/9912422 [00:00<00:00, 99437018.51it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 58434970.49it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 30922832.09it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 15425529.37it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch 1/5, Training Loss: 0.1556419895930605
Epoch 2/5, Training Loss: 0.04574345427191457
Epoch 3/5, Training Loss: 0.03186318325471438
Epoch 4/5, Training Loss: 0.02185623975061595
Epoch 5/5, Training Loss: 0.0176332447713641
Training complete.


In [ ]:
import torch
from torch.utils.data import DataLoader, random_split, Subset
from torch.utils.data.dataset import random_split
from sklearn.model_selection import KFold
import torchvision.transforms as transforms
from torchvision.datasets import MNIST

# Assuming you have a PyTorch dataset (MNIST in this example)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
mnist_dataset = MNIST(root='./data', train=True, transform=transform, download=True)

# Set the number of folds
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

for fold, (train_indices, val_indices) in enumerate(kf.split(mnist_dataset)):
    print(f"Fold {fold + 1}/{num_folds}")

    # Create data loaders for training and validation
    train_dataset = Subset(mnist_dataset, train_indices)
    val_dataset = Subset(mnist_dataset, val_indices)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)



    # Define your model, optimizer, and loss function
    # model = ...
    # optimizer = ...
    # criterion = ...

    # Training loop
    num_epochs = 5
    for epoch in range(num_epochs):
        model.train()
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

        # Validation loop
        model.eval()
        with torch.no_grad():
            total_loss = 0.0
            correct = 0
            total = 0

            for inputs, targets in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                total_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()

            average_loss = total_loss / len(val_loader)
            accuracy = correct / total

            print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {average_loss}, Accuracy: {accuracy * 100:.2f}%")

    print("\n")

# After training on all folds, you can evaluate the model's overall performance or perform other tasks.


Fold 1/5
Epoch 1/5, Validation Loss: 0.0161185667878494, Accuracy: 99.46%
Epoch 2/5, Validation Loss: 0.020146102601098934, Accuracy: 99.31%
Epoch 3/5, Validation Loss: 0.016303761175491826, Accuracy: 99.52%
Epoch 4/5, Validation Loss: 0.020477583818812994, Accuracy: 99.44%
Epoch 5/5, Validation Loss: 0.01716038114072267, Accuracy: 99.49%


Fold 2/5
Epoch 1/5, Validation Loss: 0.008750520539764538, Accuracy: 99.62%
Epoch 2/5, Validation Loss: 0.009595420062476266, Accuracy: 99.67%
Epoch 3/5, Validation Loss: 0.011053679421568999, Accuracy: 99.66%
Epoch 4/5, Validation Loss: 0.009130321026822593, Accuracy: 99.66%
Epoch 5/5, Validation Loss: 0.006407821414332496, Accuracy: 99.79%


Fold 3/5
Epoch 1/5, Validation Loss: 0.00369500464106075, Accuracy: 99.88%
Epoch 2/5, Validation Loss: 0.0028442484647060984, Accuracy: 99.90%
Epoch 3/5, Validation Loss: 0.003654712354916604, Accuracy: 99.88%
Epoch 4/5, Validation Loss: 0.016427748200237957, Accuracy: 99.64%
Epoch 5/5, Validation Loss: 0.0069

In [ ]:
for fold, (train_indices, val_indices) in enumerate(kf.split(mnist_dataset)):
    print(f"Fold {fold + 1}/{num_folds}")

    # Create data loaders for training and validation
    train_dataset = Subset(mnist_dataset, train_indices)
    print(len(train_dataset))
    val_dataset = Subset(mnist_dataset, val_indices)
    print(len(val_dataset))

Fold 1/5
48000
12000
Fold 2/5
48000
12000
Fold 3/5
48000
12000
Fold 4/5
48000
12000
Fold 5/5
48000
12000
